In [1]:
from __future__ import print_function
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import sys
import os
import math
import base64
from time import gmtime, strftime
import random, string
import time
import psutil
import base64
import uuid
import socket
import os
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import torch
from torchvision import transforms
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from pymilvus import MilvusClient
import os
from IPython.display import display



/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from __future__ import print_function
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import sys
import os
import math
import base64
from time import gmtime, strftime
import random, string
import time
import psutil
import base64
import uuid
import socket
import os
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import torch
from torchvision import transforms
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from pymilvus import MilvusClient
import os
from IPython.display import display

In [3]:
# -----------------------------------------------------------------------------

class FeatureExtractor:
    def __init__(self, modelname):
        # Load the pre-trained model
        self.model = timm.create_model(
            modelname, pretrained=True, num_classes=0, global_pool="avg"
        )
        self.model.eval()

        # Get the input size required by the model
        self.input_size = self.model.default_cfg["input_size"]

        config = resolve_data_config({}, model=modelname)
        # Get the preprocessing function provided by TIMM for the model
        self.preprocess = create_transform(**config)

    def __call__(self, imagepath):
        # Preprocess the input image
        input_image = Image.open(imagepath).convert("RGB")  # Convert to RGB if needed
        input_image = self.preprocess(input_image)

        # Convert the image to a PyTorch tensor and add a batch dimension
        input_tensor = input_image.unsqueeze(0)

        # Perform inference
        with torch.no_grad():
            output = self.model(input_tensor)

        # Extract the feature vector
        feature_vector = output.squeeze().numpy()

        return normalize(feature_vector.reshape(1, -1), norm="l2").flatten()

In [8]:
extractor = FeatureExtractor("resnet34")

from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

from dotenv import load_dotenv
load_dotenv()

# OR, the same with increased verbosity
load_dotenv(verbose=True)

# OR, explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

DOWNLOAD_DIR = "/opt/demo/images"
DIMENSION = 512 
MILVUS_URL = "https://in05-7bd87b945683c8d.serverless.gcp-us-west1.cloud.zilliz.com" 
COLLECTION_NAME = "rpipose"
TOKEN = os.environ.get("ZILLIZ_TOKEN")

# Milvus Lite
# milvus_client = MilvusClient(uri="pipose.db")

# Cloud Server
milvus_client = MilvusClient( uri=MILVUS_URL, token=TOKEN )

slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)

search_image_name = DOWNLOAD_DIR + "/tim.jpg"


In [9]:
# -----------------------------------------------------------------------------
# Search Milvus for that vector and filter by a label
# -----------------------------------------------------------------------------
results = milvus_client.search(
    COLLECTION_NAME,
    data=[extractor(search_image_name)],
    filter='label in ["person"]',
    output_fields=["label", "confidence", "id", "lefteye", "righteye"],
    search_params={"metric_type": "COSINE"},
    limit=5
)

# -----------------------------------------------------------------------------
# Iterate through last five results and display metadata and image
# -----------------------------------------------------------------------------
for result in results:
    for hit in result[:5]:
        label = hit["entity"]["label"]
        confidence = hit["entity"]["confidence"]
        righteye = hit["entity"]["righteye"]
        lefteye = hit["entity"]["lefteye"]

        print(f"Detection: {label} {confidence:.2f} Left Eye: {lefteye}, Right Eye: {righteye}" )

        try:
            response = client.chat_postMessage(
                channel="C06NE1FU6SE",
                text=(f"Search Result Detection: {label} {confidence:.2f}")
            )
        except SlackApiError as e:
            # You will get a SlackApiError if "ok" is False
            assert e.response["error"]

        try:
            response = client.chat_postMessage(
                channel="C06NE1FU6SE",
                text=(f" Search Result Left eye: {lefteye} Right eye: {righteye} ")
            )
        except SlackApiError as e:
            # You will get a SlackApiError if "ok" is False
            assert e.response["error"]

        

Detection: person 0.79 Left Eye: x: 437.00 y: 138.00, Right Eye: x: 381.00 y: 124.00
Detection: person 0.61 Left Eye: x: 284.00 y: 142.00, Right Eye: x: 219.00 y: 114.00
Detection: person 0.77 Left Eye: x: 232.00 y: 128.00, Right Eye: x: 158.00 y: 128.00
Detection: person 0.75 Left Eye: x: 232.00 y: 128.00, Right Eye: x: 158.00 y: 128.00
Detection: person 0.81 Left Eye: x: 344.00 y: 188.00, Right Eye: x: 265.00 y: 188.00
